In [17]:
# Enter your code here. Read input from STDIN. Print output to STDOUT

import binascii
import struct
import socket

printables_txt = b'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ '
backslash_txt = b'"'

printables_label = b'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~.'
backslash_label = b'.\\'

QUERY = 0  
    
_by_text = { 
    'QUERY': QUERY,  
 } 

_by_value = dict((y, x) for x, y in _by_text.items()) 

def to_text(value): 
    text = _by_value.get(value) 
    if text is None: 
        text = str(value) 
    return text

def name_from_wire_message(msg, offset):
    labels, offset = get_name_labels(msg, offset, [])
    return Name(labels), offset


def get_name_labels(msg, offset, c_offset_list):

    labellist = []
    Done = False
    while not Done:
        llen, = struct.unpack('B', msg[offset:offset+1])
        if (llen >> 6) == 0x3:                
            c_offset = struct.unpack('!H', msg[offset:offset+2])[0] & 0x3fff
            if c_offset in c_offset_list:
                raise Exception("Found compression pointer loop.")
            c_offset_list.append(c_offset)
            offset += 2
            rightmostlabels, _ = get_name_labels(msg, c_offset, c_offset_list)
            labellist += rightmostlabels
            Done = True
        else:
            offset += 1
            label = msg[offset:offset+llen]
            offset += llen
            labellist.append(label)
            if llen == 0:
                Done = True
    return (labellist, offset)

def hexdump(inputbytes):
    return binascii.hexlify(inputbytes).decode('ascii')

def bytes2escapedstring(indata, to_backslash, printables):

    out = ''
    for c in indata:
        c_chr = chr(c)
        if c in to_backslash:
            out += ("\\" + c_chr)
        elif c in printables:
            out += c_chr
        else:
            out += r"\{:03d}".format(c)
    return out

def decode_rr(pkt, offset):

    orig_offset = offset
    domainname, offset = name_from_wire_message(pkt, offset)
    rrtype, rrclass, ttl, rdlen = \
            struct.unpack("!HHIH", pkt[offset:offset+10])
    offset += 10
    rdata = pkt[offset:offset+rdlen]
    if rrtype == 1:
        rdata = socket.inet_ntop(socket.AF_INET, rdata)
    elif rrtype == 28:
        rdata = socket.inet_ntop(socket.AF_INET6, rdata)
    elif rrtype == 5:                        
        rdata, _ = name_from_wire_message(pkt, offset)      
        rdata = rdata.text()
    offset += rdlen
    return (domainname, rrtype, rrclass, ttl, rdata, offset)

class Name:
    
    labels = None

    def __init__(self, labels):
        self.labels = labels

    def text(self):
        result_list = []

        for label in self.labels:
            result = bytes2escapedstring(label,
                                         backslash_label, printables_label)
            result_list.append(result)

        if result_list == ['']:
            return "."
        return ".".join(result_list)

    def __repr__(self):
        return "<Name: {}>".format(self.text())
    
class DNSparam:

    def __init__(self, prefix, name2val):
        self.name2val = name2val
        self.val2name = dict([(y, x) for (x, y) in name2val.items()])
        self.prefix = prefix

    def get_name(self, val):
        if self.prefix:
            return self.val2name.get(val, "%s%d" % (self.prefix, val))
        return self.val2name[val]


DICT_RRTYPE = {
    "A": 1,
}


DICT_RRCLASS = {
    "IN": 1,
}


DICT_RCODE = {
    "NOERROR": 0,
}
    
qt = DNSparam("TYPE", DICT_RRTYPE)
qc = DNSparam("CLASS", DICT_RRCLASS)
rc = DNSparam("RCODE", DICT_RCODE)
    
class DNSresponse:
    sections = ["QUESTION", "ANSWER"]

    def __init__(self, msg):
        self.message = binascii.unhexlify(msg)
        self.msglen = len(self.message)
        self.decode_header()

    def decode_header(self):
        self.txid, flags, self.qdcount, self.ancount, self.nscount, \
            self.arcount = struct.unpack('!HHHHHH', self.message[:12])
        self.qr = flags >> 15
        self.opcode = (flags >> 11) & 0xf
        self.aa = (flags >> 10) & 0x1
        self.tc = (flags >> 9) & 0x1
        self.rd = (flags >> 8) & 0x1
        self.ra = (flags >> 7) & 0x1
        self.z = (flags >> 6) & 0x1
        self.ad = (flags >> 5) & 0x1
        self.cd = (flags >> 4) & 0x1
        self.rcode = (flags) & 0xf

    def print_header(self):
        print(";; ->>HEADER<<- opcode: %s, status: %s, id: %d" %
                (to_text(self.opcode), rc.get_name(self.rcode), self.txid))
        line = ";; flags:"
        for flag, val in [("qr", self.qr), ("aa", self.aa), ("tc", self.tc), ("rd", self.rd), ("ra", self.ra), ("z", self.z), ("ad", self.ad), ("cd", self.cd)]:
            if val:
                line = line + " " + flag
        line = line + ";"
        print(line + " QUERY: %d, ANSWER: %d, AUTHORITY: %d, ADDITIONAL: %d" %
                (self.qdcount, self.ancount, self.nscount, self.arcount))

    def print_rr(self, rrname, ttl, rrtype, rrclass, rdata):
        _rrtype = ""
        if qt.get_name(rrtype) == "TYPE28":
            _rrtype = "AAAA"
        elif qt.get_name(rrtype) == "TYPE5":
            _rrtype = "CNAME"
        else:
            _rrtype = qt.get_name(rrtype)
        print("%s\t\t%d\t%s\t%s\t%s" %
                (rrname.text(), ttl,
                qc.get_name(rrclass), _rrtype, rdata))
        return

    def decode_question(self, offset):
        domainname, offset = name_from_wire_message(self.message, offset)
        rrtype, rrclass = struct.unpack("!HH", self.message[offset:offset+4])
        offset += 4
        return (domainname, rrtype, rrclass, offset)

    def decode_sections(self, is_axfr=False):
        
        offset = 12                    
        answer_qname = None

        for (secname, rrcount) in zip(self.sections,
                                        [self.qdcount, self.ancount, self.nscount, self.arcount]):
            if rrcount and (not is_axfr):
                print("\n;; %s SECTION:" % secname)
            
            if secname == "QUESTION":
                for _ in range(rrcount):
                    rrname, rrtype, rrclass, offset = \
                            self.decode_question(offset)
                    _rrtype = ""
                    if qt.get_name(rrtype) == "TYPE28":
                        _rrtype = "AAAA"
                    elif qt.get_name(rrtype) == "TYPE5":
                        _rrtype = "CNAME"
                    else:
                        _rrtype = qt.get_name(rrtype)
                    answer_qname = rrname
                    if is_axfr:
                        continue
                    print(";%s\t\t%s\t%s" % (answer_qname.text(),
                                            qc.get_name(rrclass),
                                            _rrtype))
            else:
                for _ in range(rrcount):
                    rrname, rrtype, rrclass, ttl, rdata, offset = \
                            decode_rr(self.message, offset)
                    if is_axfr and (secname != "ANSWER"):
                        continue
                    else:
                        self.print_rr(rrname, ttl, rrtype, rrclass, rdata)

    def print_all(self):
        self.print_header()
        self.decode_sections()

    
if __name__ == "__main__":
    message = input()
    DNSresponse(message).print_all()

 a01d81800001000100000000076578616d706c6503636f6d0000010001c00c0001000100001bbc00045db8d822


;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 40989
;; flags: qr rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 0

;; QUESTION SECTION:
;example.com.		IN	A

;; ANSWER SECTION:
example.com.		7100	IN	A	93.184.216.34
